<ul class="breadcrumb">
  <li><a href="1.1.Univaraite_Linear_Mixed_Effects_Model.ipynb">Univariate Basics</a></li>
  <li><a href="1.2.Univariate_Linear_Additive_Genetic_Model.ipynb">Univariate Additive Genetic Model</a></li> 
  <li><a href="1.3.Univariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Univariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univaraite Linear Mixed Effects Model</font> 
</div>

In [1]:
using DataFrames
using JWAS: ST,Datasets

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMME","data.txt")
data      = readtable(phenofile,separator = ',',header=true)

,sow,site,yr,age,geneticCode,parity,nwn,SYS
1,100-113,113,2005,18,PIC 1,1,8,113_2005_WNTR
2,100-113,113,2006,18,PIC 1,2,12,113_2006_SPNG
3,100-5,5,2008,15,PIC 2,1,10,5_2008_ATMN
4,1000-5,5,2009,17,PIC 2,1,10,5_2009_SPNG
5,10000-13,13,2004,16,Commercial,1,9,13_2004_WNTR
6,10000-13,13,2004,18,Commercial,2,10,13_2004_SMMR
7,10000-13,13,2004,20,Commercial,3,11,13_2004_ATMN
8,10000-13,13,2005,18,Commercial,4,11,13_2005_SPNG
9,10000-13,13,2005,25,Commercial,5,7,13_2005_ATMN
10,10000-6,6,2012,17,PIC C27,1,12,6_2012_ATMN


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [16]:
model  = "nwn = intercept + parity + site + parity*site + yr + geneticCode + age + sow + SYS"

"nwn = intercept + parity + site + parity*site + yr + geneticCode + age + sow + SYS"

In [17]:
varRes = 2.97
mme    = build_model(model,varRes)

set_covariate(mme,"age")

ν      = 4.0      
v1     = 0.26
set_random(mme,"sow",v1,ν);

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [18]:
outputMCMCsamples(mme,"parity","age");

In [19]:
out=runMCMC(mme,data,chain_length=50000,printout_frequency=20000);

running MCMC  38%|███████████████                       |  ETA: 0:00:02
Posterior means at iteration: 20000
Residual variance: 2.943
running MCMC  77%|█████████████████████████████         |  ETA: 0:00:01
Posterior means at iteration: 40000
Residual variance: 2.979
running MCMC 100%|██████████████████████████████████████| Time: 0:00:04


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [20]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior Mean of Location Parameters"
  "MCMC samples for residual variance"
  "MCMC samples for: parity"
  "MCMC samples for: age"
  "MCMC samples for: variance of sow"

In [22]:
out["Posterior Mean of Location Parameters"]

45x2 Array{Any,2}:
 "intercept: intercept"    -2.26533    
 "parity: 1"             -135.746      
 "parity: 2"              173.275      
 "parity: 3"              130.003      
 "parity: 4"              511.708      
 "parity: 5"                6.21846    
 "site: 113"               76.2627     
 "site: 5"               -235.386      
 "site: 13"              -133.214      
 "site: 6"                -70.6333     
 "parity*site: 1 x 113"   685.497      
 "parity*site: 2 x 113"  -220.759      
 "parity*site: 1 x 5"      98.244      
 ⋮                                     
 "sow: 10000-13"            0.000818436
 "sow: 10000-6"             0.000328258
 "SYS: 113_2005_WNTR"     -64.9883     
 "SYS: 113_2006_SPNG"     466.906      
 "SYS: 5_2008_ATMN"       245.988      
 "SYS: 5_2009_SPNG"       399.359      
 "SYS: 13_2004_WNTR"       88.7244     
 "SYS: 13_2004_SMMR"      -21.7205     
 "SYS: 13_2004_ATMN"     -547.951      
 "SYS: 13_2005_SPNG"     -496.211      
 "SYS: 13_2005_ATMN" 

In [23]:
using Plots
sample4age = out["MCMC samples for: age"]
histogram(sample4age)